In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
GeForce RTX 2080 with Max-Q Design


In [3]:
# data_dir = '../../../../BottleStoodUp_atNight/Positive/'        #For the work laptop
data_dir = '../../../Images/BottleStoodUp_atNight/Positive'        #For the home laptop

In [4]:
# transform_characteristics = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor()])

transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset = datasets.ImageFolder(data_dir, transform=transform_characteristics)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

Building the network in which the pre-trained model will be loaded.

In [16]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # 32, 1, 224, 224.  Batch size, input channels, shape of the image.
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image divided by 2 approximately.   
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> 15, 110, 44, 44. Batch size, channel output, output image shape.
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # Dimension rseult: -> 15, 450, 1, 1
        )
        
        # Initial dimension for this part of the model: 15 , 450, 1, 1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(output_channels_layer3, output_channels_layer2, 8),  # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels_layer2, output_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # Dimension rseult: -> 15, 110, 44, 44
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels, input_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # Dimension rseult: -> 15, 1, 224, 224
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

In [17]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

device = get_device()
print(device)

cuda:0


In [18]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [19]:
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

Loading the model

In [20]:
filepath = "../../../BottlesAnomalies_TFM/models/pytorchModels/PytorchModel_withCUDA"
# For loading the model 
model.load_state_dict(torch.load(filepath))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

Extracting the layers' weights of the model that has been loaded.

In [21]:
layers_weights_list = []
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        weights = m.weight
        layers_weights_list.append(weights)
print("The number of layers that the loaded model has, is: ", len(layers_weights_list))

The number of layers that the loaded model has, is:  3


Copying the model layers' weights from one model to another. 

In [22]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # 32, 1, 224, 224.  Batch size, input channels, shape of the image.
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image divided by 2 approximately. 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> 15, 110, 44, 44. Batch size, channel output, output image shape.
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # Dimension rseult: -> 15, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # Dimension rseult: -> 15, 450, 1, 1
        )

    def forward(self, x):
        # print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the show modules function")
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
            
    def show_one_layer_weights(self, index):
        print("This is the one layer show function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == index:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                i = i +1
    
    def update_weights(self):
        print("updating weights function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = layers_weights_list[i]
                i = i +1

                                        
 

In [23]:
model_encoder = Autoencoder_latentSpace()
model_encoder.to(device)
# model.eval()

Autoencoder_latentSpace(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
)

In [25]:
model_encoder.update_weights()

updating weights function


In [26]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-1.6085e-25, -1.6363e-25, -1.6663e-25,  ..., -1.7430e-25,
           -1.7557e-25, -1.7903e-25],
          [-2.2771e-25, -2.5256e-25, -2.5323e-25,  ..., -2.6019e-25,
           -2.5946e-25, -2.5337e-25],
          [-2.5053e-25, -2.6664e-25, -2.5478e-25,  ..., -2.7490e-25,
           -2.6799e-25, -2.6038e-25],
          ...,
          [-2.6943e-25, -2.8397e-25, -2.8483e-25,  ..., -3.0378e-25,
           -2.9532e-25, -2.8298e-25],
          [-2.8312e-25, -2.8671e-25, -2.8712e-25,  ..., -2.9985e-25,
           -2.9239e-25, -2.9911e-25],
          [-2.9684e-25, -2.9257e-25, -2.9676e-25,  ..., -3.0312e-25,
           -3.0702e-25, -3.0488e-25]],

         [[-1.6082e-25, -1.6399e-25, -1.6598e-25,  ..., -1.7378e-25,
           -1.7550e-25, -1.7926e-25],
          [-2.2830e-25, -2.5180e-25, -2.5322e-25,  ..., -2.6058e-25,
           -2.5928e-25, -2.5301e-25],
          [-2.5050e-25, -2.6661e-25, -2.55

At this point, the created "model_encoder" model contains the weights of the encoder part of the pre-trained Autoencoder model. 

## Obtaining the KDE representation of the training set images

In [27]:
########################################################
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
# encoded_images = model_encoder(images)
encoded_images = []

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Convert "encoded_images" to a np array

In [29]:
np_encoded_images = []
for i in range (len(encoded_images)):
    # np_conversion = encoded_images[i].detach().numpy()        # If not using GPU
    np_conversion = encoded_images[i].cpu().detach().numpy()    # If using GPU
    np_encoded_images.append(np_conversion)
np_encoded_images = np.array(np_encoded_images)
print(type(np_encoded_images))

<class 'numpy.ndarray'>


In [30]:
print(np_encoded_images.shape)

(179, 450, 1, 1)


Also, see above the shape of the representation of the original images has been lowered to (1, 1) as specified in the model structure. The number 450, on the other hand, corresponds to the channels of the image; this value started at 3 and layer by layer it incremented until reaching 450.

Now, we have to flatten the data in order to apply kernel density on it.

In [31]:
model_encoder_output_shape = (450,1,1)
print(model_encoder_output_shape)

(450, 1, 1)


In [32]:
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]
encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_images]


In [35]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

The above function fits a kernel density estimation to the data that is provided, that is, to the "encoded_images_vector" variable. It does so using a Guassian kernel of bandwidth 0.2.

The badnwidth parameter affects on how the selected kernel will fit each sample of the given data. For example for the case in which the kernel is a Gaussian distribution, the bandwidth parameter would affect in how thin or wide is the Gaussian distribution.

At this point in the variable "kde" we have some numbers that are the result of fitting Gaussian functions to the given data points in the variable "encoded_images_vecotr". We will use the "kde" variable later for scoring with it, some given data points; the scoring will be given depending on how similar are the given data points to the ones that it had estimated.

Here below, it is shown the kde values corresponding to each encoded sample:

In [36]:
density_vals = kde.score_samples(encoded_images_vector)
print(density_vals)

[305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048203 306.23048203 306.2304823  306.2304823  306.23048203
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048

Notice that the above density values are pretty much the same among them. This has to do with the new model that was trained. The previous model from the "Pt_latentSpace_DS1" program, makes these value to be more different among them.

From here, the mean and standard deviation of these values are computed

In [37]:
average_density = np.mean(density_vals)
stdev_density = np.std(density_vals)
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)

The avg of the density values is:  305.6924951518362
The stdev_density of the density values is:  0.28860949910524847


Now, it will be shown the density mean and std deviation of the set of anomalies samples

In [38]:
data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the home laptop
# data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the work laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset_anomalies = datasets.ImageFolder(data_anomalies, transform=transform_characteristics)
dataloader_anomalies = torch.utils.data.DataLoader(dataset_anomalies, batch_size=32, shuffle=True)

In [39]:
#Get encoded output of input images = Latent space
encoded_anomalies_images = []

for i in range(len(dataset_anomalies)):
    X = dataset_anomalies[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_anomalies_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [40]:
np_encoded_anomaly_images = []
for i in range (len(encoded_anomalies_images)):
    # np_conversion = encoded_anomalies_images[i].detach().numpy()      # If GPU is not used
    np_conversion = encoded_anomalies_images[i].cpu().detach().numpy()    # If GPU is used
    np_encoded_anomaly_images.append(np_conversion)
np_encoded_anomaly_images = np.array(np_encoded_anomaly_images)
print(type(np_encoded_anomaly_images))

<class 'numpy.ndarray'>


In [41]:
encoded_anomaly_images_vector = [np.reshape(img_encoded, (out_vector_shape)) for img_encoded in np_encoded_anomaly_images]


In [42]:
density_vals_anomalies = kde.score_samples(encoded_anomaly_images_vector)
print(density_vals_anomalies)

[298.71892374  62.53676635 224.54893745 283.88478455 280.10385174
 238.14052316]


In [43]:
average_density_anomalies = np.mean(density_vals_anomalies)
stdev_density_anomalies = np.std(density_vals_anomalies)
print("The avg of the density values is: ", average_density_anomalies)
print("The stdev_density of the density values is: ", stdev_density_anomalies)

The avg of the density values is:  231.3222978307193
The stdev_density of the density values is:  79.85996331218182


See that the std deviation along with the mean of these density values will overlap the mean of the non-anomaly images. 

Considering the situation of the overlapping commented above, given a density value, it will be built a function that:
- Assigns a percentage value according to the proximity to the mean of the non-anomaly images. For example: 
    - If the densitiy value is 305.69 (the mean of the density values of the non-anomaly images), then this density value should have 100% chance to be considered as non-anomaly.
    - If the densitiy value is 305.4014 (the mean of the density values of the non-anomaly minus the std deviation of the same set), then this density value should have 50% chance to be considered as non-anomaly.
- Assigns a percentage value according to the proximity to the mean of the anomaly images. For example: 
    - If the densitiy value is 231.6065 (the mean of the density values of the anomaly images), then this density value should have 100% chance to be considered as an anomaly image.
    - If the densitiy value is 311.0713 (the mean of the density values of the anomaly plus the std deviation of the same set), then this density value should have 50% chance to be considered as an anomaly.
- The two percentage values from above will be summed up assigning the following weights to the equation:

        = perc_NOanomaly*0.75 + perc_anomaly*0.25
        
    More weight is assigned to the non-anomaly images because there are more samples of this kind of images.
- After the weighted sum, the result will be subtracted from 100, to finally output the probability of an image to be an anomaly image. 

In [54]:
def ranges_mapper(value, leftMin, leftMax, rightMin, rightMax):
    # if(value>leftMax):
    #     return rightMax
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [70]:
def map_kde2prob_list(input_list):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images
    prob_score_list = []

    for i in range (len(input_list)):
        score_NOanomaly = input_list[i] - threshold_NOanomaly

        perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
        if perc_NOanomaly<0:
            perc_NOanomaly = 0
        if perc_NOanomaly>100:
            perc_NOanomaly = 100
        

        score_anomaly = input_list[i] - threshold_anomaly

        perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
        if perc_anomaly<0:
            perc_anomaly = 0
        if perc_anomaly>100:
            perc_anomaly = 100
        
        prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
        prob_score_list.append(prob_score)
    return prob_score_list

In [71]:
def map_kde2prob(value):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images          # The std deviation of the density values corresponding to the anomaly images

    score_NOanomaly = value - threshold_NOanomaly

    perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
    if perc_NOanomaly<0:
        perc_NOanomaly = 0
    if perc_NOanomaly>100:
        perc_NOanomaly = 100
    # print(perc_NOanomaly)

    score_anomaly = value - threshold_anomaly

    perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
    if perc_anomaly<0:
        perc_anomaly = 0
    if perc_anomaly>100:
        perc_anomaly = 100
    # print(perc_anomaly)
    prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
  
    return prob_score

In [72]:
def computePred(kde_value):
    pred = 0
    prob_anomaly = map_kde2prob(kde_value)/100
    if prob_anomaly > 0.5:
        pred = 1
    return pred

In [74]:
print("The prob of the kde value of being anomaly image is: ", map_kde2prob(average_density+stdev_density))
print("Given the probability, it is actually predicted as:", computePred(average_density+stdev_density))

The prob of the kde value of being anomaly image is:  49.18589423970193
Given the probability, it is actually predicted as: 0


Testing the kde thresholds 

In [75]:
test_dir = '../../../Images/BottleStoodUp_atNight/Evaluation'      #This is for the home laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])

dataset_test = datasets.ImageFolder(test_dir, transform=transform_characteristics)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=True)
classes = ('non-anomaly','anomaly')

In [76]:
#Get encoded output of input images = Latent space
encoded_test_imgs = []


for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()     
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    # np_conversion = Y.detach().numpy()
    np_conversion = Y.cpu().detach().numpy()
    encoded_test_imgs.append(np_conversion)
np_encoded_test_images = np.array(encoded_test_imgs)
print(type(np_encoded_test_images))

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


<class 'numpy.ndarray'>


In [77]:
encoded_test_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_test_images]
print(len(encoded_test_images_vector))

30


In [78]:
density_vals_test = kde.score_samples(encoded_test_images_vector)
print(density_vals_test)

[  -350.41798728   -692.88343352  -2021.30365965  -2182.68227401
  -2798.86127057  -3264.94163834  -5160.29215237  -2769.45912666
  -1146.9199288   -2419.104461      117.12551799    146.73382904
    269.89463124    -79.90776409   -564.56551367     74.16671941
 -20734.51477073 -16402.92998568  -2100.0954696    -265.96934953
   -235.60839811   -541.94455948   -659.56090768   -559.52017019
   -613.11101841   -952.02201969   -920.03061571  -1473.07419319
  -1754.96438068  -1134.19929984]


The above shown values are REALLY strange. It was not expected to have negative values in the density numbers!

In [79]:
prob_test = map_kde2prob_list(density_vals_test)

In [80]:
print(prob_test)

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 92.87453548429325, 88.24012453805074, 81.03749548097213, 100.0, 100.0, 99.59861799061724, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [81]:
y_pred = []
y_true = []

# Grabbing only the first image of the anomalies dataset
X = dataset_test[0]
image_in_tensor = X[0]
 
n_features = len(image_in_tensor[0])  # Get the size of one image of the anomaly images dataset. This is supposed to be 224
for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda() 
    ground_truth = X[1]

    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_converted_encoded_img = Y.cpu().detach().numpy()
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    prediction = computePred(density)
    y_pred.append(prediction) # Save Prediction
    y_true.append(ground_truth) # Save Truth
print(y_true)
print(y_pred)

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


Above, it can be seen that all the images are predicted to be anomaly images. This is good for the TPR but it is terrible for the FPR metric.